### 1. Download W&B results

In [1]:
from olmo.scaling.scaling_laws.download_wandb_logs import main as download_wb
from argparse import Namespace

/Users/akshitab/virtuals/olmo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
run_names = [
    "ai2-llm/olmo-ladder/amberish-150M-1xC",
    "ai2-llm/olmo-ladder/amberish-300M-1xC",
    "ai2-llm/olmo-ladder/amberish-750M-1xC",
    "ai2-llm/olmo-ladder/amberish-1B-1xC"
]

validation_outputs = {}
downstream_outputs = {}

for run_name in run_names:

    output_name = run_name.split("/")[-1]
    output_path = f"wandb/{output_name}-all-val.csv"
    args = Namespace(
        wandb_names=[run_name],
        x_axis="throughput/total_training_Gflops",
        y_axis=["eval/all-validation/CrossEntropyLoss"],
        output_path=output_path,
    )
    download_wb(args)
    validation_outputs[output_name] = output_path

    output_path = f"wandb/{output_name}-all-downstream.csv"
    args = Namespace(
        wandb_names=[run_name],
        x_axis="throughput/total_training_Gflops",
        y_axis=["eval/downstream/all"],
        output_path=output_path,
    )
    download_wb(args)
    downstream_outputs[output_name] = output_path

Found 2 matching runs in ai2-llm/olmo-ladder
 <Run ai2-llm/olmo-ladder/apwh1wq9 (finished)>
<Run ai2-llm/olmo-ladder/jlq83qfw (failed)>


  0%|                                                                                                                            | 0/2 [00:00<?, ?it/s]

Processing amberish-150M-1xC


 50%|██████████████████████████████████████████████████████████                                                          | 1/2 [00:00<00:00,  2.79it/s]

Processing amberish-150M-1xC


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.82it/s]


Found 1 matching runs in ai2-llm/olmo-ladder
 <Run ai2-llm/olmo-ladder/qah1jr09 (finished)>


  0%|                                                                                                                            | 0/1 [00:00<?, ?it/s]

Processing amberish-1B-1xC


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.46it/s]


## 2. Fitting a power law to the results of the model ladder


In [25]:
import pandas as pd

In [28]:
df = pd.read_csv(args.output_path)

In [31]:
df["throughput/total_training_Gflops"].values[-1]

181940730754.81943

In [41]:
ce_loss = {}
downstream = {}
gflops = {}
for run_name, output_path in validation_outputs.items():
    df_val = pd.read_csv(output_path)
    gflops[run_name] = df_val["throughput/total_training_Gflops"].values[-1]
    
    ce_loss[run_name] = (df_val["eval/c4_en-validation/CrossEntropyLoss"].values[-1] + df_val["eval/pile-validation/CrossEntropyLoss"].values[-1]) / 2

    df_down = pd.read_csv(downstream_outputs[run_name])
    downstream[run_name] = df_down["eval/downstream/hellaswag_len_norm"].values[-1]

In [42]:
ce_loss

{'amberish-150M-1xC': 3.1409857273101807,
 'amberish-300M-1xC': 2.909250020980835,
 'amberish-750M-1xC': 2.662544012069702,
 'amberish-1B-1xC': 2.559367537498474}

In [43]:
gflops

{'amberish-150M-1xC': 4569042832.5888,
 'amberish-300M-1xC': 17627224913.750263,
 'amberish-750M-1xC': 84206674904.64223,
 'amberish-1B-1xC': 181940730754.81943}

In [44]:
downstream

{'amberish-150M-1xC': 0.2932682633399963,
 'amberish-300M-1xC': 0.335789680480957,
 'amberish-750M-1xC': 0.4260107576847076,
 'amberish-1B-1xC': 0.4780920147895813}

In [47]:
import re
run_name_re = re.compile(r"^([^-]+)-([^-]+)-([^-]+)$")
_number_unit_re = re.compile(r"^([0-9]+)([a-zA-Z]+)$")

def parse_size(size: str) -> int:
    model_size, model_size_unit = _number_unit_re.match(size.strip().upper()).groups()  # type: ignore
    model_size = int(model_size)
    if model_size_unit == "K":
        model_size *= 1000
    elif model_size_unit == "M":
        model_size *= 1000000
    elif model_size_unit == "B":
        model_size *= 1000000000
    else:
        raise ValueError(f"Could not parse model size unit '{model_size_unit}'")
    return model_size


def parse_length(length: str, model_size: int) -> int:
    length_in_tokens, length_unit = _number_unit_re.match(length.strip().upper()).groups()  # type: ignore
    length_in_tokens = int(length_in_tokens)
    if length_unit == "C" or length_unit == "XC":
        length_in_tokens *= 20 * model_size
    elif length_unit == "K":
        length_in_tokens *= 1000
    elif length_unit == "M":
        length_in_tokens *= 1000000
    elif length_unit == "B":
        length_in_tokens *= 1000000000
    elif length_unit == "T":
        length_in_tokens *= 1000000000000
    else:
        raise ValueError(f"Could not parse length '{args.length}'")
    return length_in_tokens

def parse_run_name(name: str):
    name, size, length = run_name_re.match(name).groups()
    size = parse_size(size)
    length = parse_length(length, size)
    return name, size, length

In [ ]:
import numpy as np

def power_law(x, a, b, c):
    return a * x ** b + c

power_law_bounds = ([0, -np.inf, 0], [np.inf, 0, np.inf])

from scipy.optimize import curve_fit

fix, ax1 = plt.subplots(1, figsize=(10, 5))

# model size
x = []
y = []
for run in ["baseline-150M-1xC", "baseline-300M-1xC", "baseline-530M-1xC", "baseline-750M-1xC", "baseline-1B-1xC"]:
#for run in ["baseline-150M-1xC", "baseline-300M-1xC", "baseline-750M-1xC", "baseline-1B-1xC"]:
    _, size, _ = run_name_re.match(run).groups()
    _, size = ladder.size_for_model(size)
    #x.append(size)
    x.append(gflops[run])
    y.append((pile_ce[run] + c4_ce[run])/2)

train_x = np.array(x[:-1])
train_y = np.array(y[:-1])
popt, pcov = curve_fit(power_law, train_x, train_y, p0=(100, -0.2, 1.5), bounds=power_law_bounds, max_nfev=50000)
print(popt)
ax1.scatter(x, y)
ax1.set_title("Varying flops, all Cx1")
ax1.set_xlabel("GFlops")
ax1.set_ylabel("CE on Pile and C4")
x = np.linspace(min(x), max(x), 100)
ax1.plot(x, power_law(x, *popt))

pred_plots = power_law(x, *popt)

print("x: ", train_x)
print("actual y", y)
print("chinchilla fit", popt)
print("predicted y at 1b", pred_plots[-1])

plt.tight_layout()
plt.show()